In [1]:
from langchain.llms import OpenAI
from langchain.tools.python.tool import PythonAstREPLTool
from langchain.schema import AgentAction, AgentFinish
from langchain.schema.messages import BaseMessage, SystemMessage, AIMessage
from langchain.agents.openai_functions_agent.base import OpenAIFunctionsAgent, _format_intermediate_steps, _FunctionsAgentAction
from langchain.callbacks.manager import Callbacks
from langchain.agents.agent import AgentExecutor

from typing import Union, TypeVar, List, Tuple, Any
import json
from json import JSONDecodeError

class CustomPythonAstREPLTool(PythonAstREPLTool):
    name = "python"
    description = (
        "A Python shell. Use this to execute python commands. "
        "The input must be an object as follows: "
        "{'__arg1': 'a valid python command.'} "
        "When using this tool, sometimes output is abbreviated - "
        "Make sure it does not look abbreviated before using it in your answer. "
        "Don't add comments to your python code."
    )

def _parse_ai_message(message: BaseMessage) -> Union[AgentAction, AgentFinish]:
    """Parse an AI message."""
    if not isinstance(message, AIMessage):
        raise TypeError(f"Expected an AI message got {type(message)}")

    function_call = message.additional_kwargs.get("function_call", {})

    if function_call:
        function_call = message.additional_kwargs["function_call"]
        function_name = function_call["name"]
        try:
            _tool_input = json.loads(function_call["arguments"])
        except JSONDecodeError:
            print(
                f"Could not parse tool input: {function_call} because "
                f"the `arguments` is not valid JSON."
            )
            _tool_input = function_call["arguments"]

        # HACK HACK HACK:
        # The code that encodes tool input into Open AI uses a special variable
        # name called `__arg1` to handle old style tools that do not expose a
        # schema and expect a single string argument as an input.
        # We unpack the argument here if it exists.
        # Open AI does not support passing in a JSON array as an argument.
        if "__arg1" in _tool_input:
            tool_input = _tool_input["__arg1"]
        else:
            tool_input = _tool_input

        content_msg = "responded: {content}\n" if message.content else "\n"

        return _FunctionsAgentAction(
            tool=function_name,
            tool_input=tool_input,
            log=f"\nInvoking: `{function_name}` with `{tool_input}`\n{content_msg}\n",
            message_log=[message],
        )

    return AgentFinish(return_values={"output": message.content}, log=message.content)

class CustomOpenAIFunctionsAgent(OpenAIFunctionsAgent):
    def plan(
        self,
        intermediate_steps: List[Tuple[AgentAction, str]],
        callbacks: Callbacks = None,
        **kwargs: Any,
    ) -> Union[AgentAction, AgentFinish]:
        """Given input, decided what to do.
        Args:
            intermediate_steps: Steps the LLM has taken to date, along with observations
            **kwargs: User inputs.
        Returns:
            Action specifying what tool to use.
        """
        user_input = kwargs["input"]
        agent_scratchpad = _format_intermediate_steps(intermediate_steps)
        prompt = self.prompt.format_prompt(
            input=user_input, agent_scratchpad=agent_scratchpad
        )
        messages = prompt.to_messages()
        predicted_message = self.llm.predict_messages(
            messages, functions=self.functions, callbacks=callbacks
        )
        agent_decision = _parse_ai_message(predicted_message)
        return agent_decision
    

In [2]:
import pandas as pd

df_exposure = pd.read_csv("fake port - Exposure.csv")
df_perf = pd.read_csv("fake port - Perf.csv")
len(df_exposure) , len(df_perf)

(50, 50)

In [4]:
from langchain.chat_models import ChatOpenAI
import openai

from langchain.agents.agent import AgentExecutor

import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

# openai.api_key  = os.environ['OPENAI_API_KEY']
openai.api_key= os.environ['OPENAI_API_KEY']

llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")

In [5]:
prefix = 'You are a portfolio engineer who analyze stock portfolios. You are given 2 portfolio dataframes, df_exposure and df_perf to analyze.'
# prefix=""

dataset={"df_exposure": df_exposure,
         "df_perf": df_perf
         }
tools = [CustomPythonAstREPLTool(locals=dataset)]
tool_names = [tool.name for tool in tools]
prompt = CustomOpenAIFunctionsAgent.create_prompt(system_message=SystemMessage(content=prefix))
agent = AgentExecutor.from_agent_and_tools(
    agent=CustomOpenAIFunctionsAgent(llm=llm, prompt=prompt, tools=tools, verbose=True),
    tools=tools, 
    verbose=True
)

### prompt1: compare 2 dataframes.

In [6]:
res=agent.run("""how many rows does each df have? Output the result in JSON format
              eg
              {
              "df_exposure": [row count],
              "df_perf": [row count]    
              }
              """)
res



> Entering new  chain...

Invoking: `python` with `df_exposure.shape[0]`


50
Invoking: `python` with `df_perf.shape[0]`


50{
  "df_exposure": 50,
  "df_perf": 50
}

> Finished chain.


'{\n  "df_exposure": 50,\n  "df_perf": 50\n}'

In [7]:
res=agent.run("""what columns does each df have? 
              Output the result in JSON format
              eg
              {
              "df_exposure": [column list],
              "df_perf": [column list]    
              }
              """)
res



> Entering new  chain...


Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-b9LeZA4q56k2CbxEtZPS9Sdj on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 2.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-b9LeZA4q56k2CbxEtZPS9Sdj on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit ht


Invoking: `python` with `df_exposure.columns.tolist()`


['Date', 'PCode', 'Stock', 'Sector', 'Weight']

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-b9LeZA4q56k2CbxEtZPS9Sdj on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 2.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-b9LeZA4q56k2CbxEtZPS9Sdj on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit ht


Invoking: `python` with `df_perf.columns.tolist()`


['Date', 'PCode', 'Stock', 'Return']

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-b9LeZA4q56k2CbxEtZPS9Sdj on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 2.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-b9LeZA4q56k2CbxEtZPS9Sdj on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit ht

{
  "df_exposure": ["Date", "PCode", "Stock", "Sector", "Weight"],
  "df_perf": ["Date", "PCode", "Stock", "Return"]
}
The columns in the `df_exposure` dataframe are: "Date", "PCode", "Stock", "Sector", "Weight".

The columns in the `df_perf` dataframe are: "Date", "PCode", "Stock", "Return".

> Finished chain.


'{\n  "df_exposure": ["Date", "PCode", "Stock", "Sector", "Weight"],\n  "df_perf": ["Date", "PCode", "Stock", "Return"]\n}\nThe columns in the `df_exposure` dataframe are: "Date", "PCode", "Stock", "Sector", "Weight".\n\nThe columns in the `df_perf` dataframe are: "Date", "PCode", "Stock", "Return".'

### Prompt2: top holdings  that requires sorting the table

In [ ]:
# res = agent.run(""" Do not create a new dataframe. 
#           Join df_exposure and df_perf by `Date`, `PCode`, `Stock`, and name the result df_merged.
#           Please comment the top holdings with the highest weights in t in table format. 
#           """)

# res

In [10]:

# res = agent.run(""" Do not create a new dataframe. 
#           Join df_exposure and df_perf by `Date`, `PCode`, `Stock`, and name the result df_merged.
#           Please comment the top holdings with the highest weights in t in table format. 
#           """)

# res



> Entering new  chain...

Invoking: `python` with `df_merged = df_exposure.merge(df_perf, on=['Date', 'PCode', 'Stock'])`




> Finished chain.


''

### Prompt3: using the df_exposure to find largest sector exposure

In [14]:
agent.run(""" 
          Uisng the df, please aggregate Weight by Sector and sort aggregated weights. 
          Finally show the sector weights in table format. 
          """)



> Entering new  chain...


Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised ServiceUnavailableError: The server is overloaded or not ready yet..



Invoking: `python` with `portfolio_df.groupby('Sector')['Weight'].sum().sort_values(ascending=False).reset_index().rename(columns={'Weight': 'Aggregated Weight'})`


                   Sector  Aggregated Weight
0        CONSUMER STAPLES              22.44
1                 FINANCE              18.61
2             HEALTH CARE              10.62
3  CONSUMER DISCRETIONARY               8.27
4          COMMUNICATIONS               3.48
5               MATERIALS               1.41
6  INFORMATION TECHNOLOGY               0.99Here is the aggregated weight by sector, sorted in descending order:

|       Sector      | Aggregated Weight |
|-------------------|-------------------|
| CONSUMER STAPLES  |       22.44       |
|      FINANCE      |       18.61       |
|    HEALTH CARE    |       10.62       |
| CONSUMER DISCRETIONARY |     8.27       |
|   COMMUNICATIONS  |       3.48        |
|     MATERIALS     |       1.41        |
| INFORMATION TECHNOLOGY |   0.99        |

> Finished chain.


'Here is the aggregated weight by sector, sorted in descending order:\n\n|       Sector      | Aggregated Weight |\n|-------------------|-------------------|\n| CONSUMER STAPLES  |       22.44       |\n|      FINANCE      |       18.61       |\n|    HEALTH CARE    |       10.62       |\n| CONSUMER DISCRETIONARY |     8.27       |\n|   COMMUNICATIONS  |       3.48        |\n|     MATERIALS     |       1.41        |\n| INFORMATION TECHNOLOGY |   0.99        |'

### Prompt4: best and worst performing clusters that requires joining and sorting

In [12]:
res = agent.run(""" 
            Join df_exposure and df_perf by `Date`, `PCode`, `Stock`, and name the result df_merged.
             
          Based off df_merged, please aggregate Return by Sector
          Then sort aggregated returns.
          Convert the aggregated returns to percent format.
          Finally show the aggregated returns in table format. 
          """)



> Entering new  chain...

Invoking: `python` with `import pandas as pd

# Join df_exposure and df_perf by `Date`, `PCode`, `Stock`, and name the result df_merged.
df_merged = pd.merge(df_exposure, df_perf, on=['Date', 'PCode', 'Stock'])

# Aggregate Return by Sector
aggregated_returns = df_merged.groupby('Sector')['Return'].sum().reset_index()

# Sort aggregated returns
aggregated_returns = aggregated_returns.sort_values('Return', ascending=False)

# Convert the aggregated returns to percent format
aggregated_returns['Return'] = aggregated_returns['Return'] * 100

# Show the aggregated returns in table format
aggregated_returns`


                   Sector      Return
2        CONSUMER STAPLES  315.372208
4             HEALTH CARE  182.948921
1  CONSUMER DISCRETIONARY   92.635406
0          COMMUNICATIONS   53.773642
6               MATERIALS   35.595362
5  INFORMATION TECHNOLOGY   13.783619
3                 FINANCE    8.124072The aggregated returns by sector, sorted in descending o

In [13]:
res

'The aggregated returns by sector, sorted in descending order, and converted to percent format are as follows:\n\n|       Sector      |   Return   |\n|-------------------|------------|\n| CONSUMER STAPLES  |  315.3722  |\n|    HEALTH CARE    |  182.9489  |\n| CONSUMER DISCRETIONARY |  92.6354  |\n|  COMMUNICATIONS   |  53.7736  |\n|     MATERIALS     |  35.5954  |\n| INFORMATION TECHNOLOGY |  13.7836  |\n|      FINANCE      |   8.1241  |'

In [6]:
agent.run(""" Do not create a new dataframe. 
          Only use portfolio_df, please aggregate Return by Sector, which is to sum up Stock weight times Stock return by Sector.
          Then sort aggregated returns.
          Convert the aggregated returns to percent format.
          Finally show the aggregated returns in table format. 
          """)



> Entering new  chain...

Invoking: `python` with `portfolio_df.groupby('Sector')['Return'].apply(lambda x: (x * portfolio_df.loc[x.index, 'Weight']).sum()).sort_values(ascending=False).map('{:.2%}'.format).reset_index().rename(columns={'Return': 'Aggregated Return'})`


NameError: name 'portfolio_df' is not definedApologies, but I need the portfolio dataframe in order to perform the analysis. Could you please provide the portfolio dataframe?

> Finished chain.


'Apologies, but I need the portfolio dataframe in order to perform the analysis. Could you please provide the portfolio dataframe?'